# W3 Assignment Part 3

## Segmenting and Clustering Neighborhoods in Toronto

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


### Explore & Transform Data

In [5]:
#Tranform the data into a pandas dataframe
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find_all(class_="wikitable sortable")[0]
rows = table.find_all('tr')
headers = [th.get_text() for th in table.find_all('th')]
df = pd.DataFrame(columns=headers, index=range(0,len(rows)))

row_marker = 0
for row in rows:
    column_marker = 0
    columns = row.find_all('td')
    for col in columns:       
        df.iloc[row_marker,column_marker]=col.get_text()
        column_marker += 1
    row_marker += 1

#Data Cleaning
df.rename(columns={'Postcode' : 'PostalCode', 'Neighbourhood\n':'Neighborhood'}, inplace=True)  #refine headers
df.dropna(axis=0, inplace=True) #drop any NaN in table
df['Neighborhood'] = df['Neighborhood'].apply(lambda x: x.strip()) #get rid of '\n'
df = df[df['Borough']!='Not assigned'].reset_index(drop=True)  #drop any data containing not-assigned-Borough
index = df.index[df['Neighborhood'] == 'Not assigned'].tolist()  #replace not-assigned-neighborhood using borough's name
df.iloc[index[0], 2] = df.iloc[index[0], 1]

geo=pd.read_csv("https://cocl.us/Geospatial_data")
geo.rename(columns={'Postal Code':'PostalCode'}, inplace = True)
df = pd.merge(df, geo, on='PostalCode')
print(df.shape)
df.head(10)

(211, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),len(df['Neighborhood'].unique())))

The dataframe has 11 boroughs and 209 neighborhoods.


### Create a map of Toronto with neighborhoods superimposed on top

In [11]:
#Use geopy library to get the latitude and longitude values of Toronto City.
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [12]:
# create map of Toronto using latitude and longitude values
map_trt = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, pcode in zip(df['Latitude'], df['Longitude'], df['Borough'], df['PostalCode']):
    label = '{}, {}'.format(pcode, borough)
    label = folium.Popup(label, parse_html=True)  #jane: parse_html=True if the popup is a template that needs to the rendered first.
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trt)  
    
map_trt

#### Let's simplify the above map and segment and cluster only the neighborhoods in 'Downtown Toronto'

In [13]:
trt_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
print(trt_data.shape) 
trt_data.head()

(37, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [15]:
# create map of Central Toronto using latitude and longitude values
map_ct = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(trt_data['Latitude'], trt_data['Longitude'], trt_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ct)  
    
map_ct

### Define Foursquare Credentials and Version

In [16]:
# The code was removed by Watson Studio for sharing.

### Explore Neighborhoods in Downtown Toronto

In [17]:
#function to explore Neighborhoods in Downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's get the top 100 venues that are in each Neighborhood in Downtown Toronto within a radius of 500 meters.

In [18]:
ct_venues = getNearbyVenues(names=trt_data['Neighborhood'],
                                   latitudes=trt_data['Latitude'],
                                   longitudes=trt_data['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [19]:
print(ct_venues.shape)
ct_venues.head()

(2490, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [20]:
ct_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,13,13,13,13,13,13
Berczy Park,55,55,55,55,55,55
CN Tower,13,13,13,13,13,13
Cabbagetown,43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
Chinatown,100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,89,89,89,89,89,89


In [21]:
print('There are {} uniques categories.'.format(len(ct_venues['Venue Category'].unique())))

There are 204 uniques categories.


### Analyze Each Neighborhood

In [22]:
# one hot encoding
ct_onehot = pd.get_dummies(ct_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ct_onehot['Neighborhood'] = ct_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ct_onehot.columns[-1]] + list(ct_onehot.columns[:-1])  #jane: last column + [column 0 until column (last-1)]
ct_onehot = ct_onehot[fixed_columns]

ct_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
ct_onehot.shape

(2490, 204)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
ct_grouped = ct_onehot.groupby('Neighborhood').mean().reset_index()   
ct_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Adelaide,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.02,0.0000,0.000000,0.0000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.060000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.00,0.010000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.000000,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.010000,0.010000,0.000000,0.010000,0.020000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.010000,0.000000,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.00000

In [25]:
ct_grouped.shape

(36, 204)

Let's build a dataframe of each neighborhood along with the top 10 most common venues

In [26]:
def return_most_common_venues(row, num_top_venues):   #jane: access ct_grouped table's column like a row
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):  #jane: value 0 until 9
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) #jane: ind 0 until 2
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) #jane: ind 3 onwards

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ct_grouped['Neighborhood']

for ind in np.arange(ct_grouped.shape[0]):  #jane: 40 values
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ct_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Bakery,Burger Joint,Gym,Hotel,Sushi Restaurant
1,Bathurst Quay,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
2,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Seafood Restaurant,Farmers Market,Steakhouse,Restaurant,Pub,Bakery
3,CN Tower,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
4,Cabbagetown,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Café,Pizza Place,Pub,Liquor Store,Japanese Restaurant,Jewelry Store


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [28]:
# set number of clusters
kclusters = 5

ct_grouped_clustering = ct_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ct_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 4, 1, 4, 4, 3, 0, 3, 4], dtype=int32)

In [29]:
print(kmeans.labels_.shape)  #jane
set(kmeans.labels_)  #jane

(36,)


{0, 1, 2, 3, 4}

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ct_merged = trt_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ct_merged = ct_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ct_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,4,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Restaurant
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,4,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Restaurant
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Lingerie Store,Pizza Place,Bar
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Lingerie Store,Pizza Place,Bar
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Coffee Shop,Restaurant,Café,Breakfast Spot,Hotel,Bakery,Italian Restaurant,Gastropub,Park,Diner


Finally, let's visualize the resulting clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ct_merged['Latitude'], ct_merged['Longitude'], ct_merged['Neighborhood'],ct_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1 (Grocery Stores / Convenience Stores)

In [34]:
ct_merged.loc[ct_merged['Cluster Labels'] == 0, ct_merged.columns[[1] + list(range(5, ct_merged.shape[1]))]]  

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Grocery Store,Café,Park,Diner,Convenience Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub


#### Cluster 2 (Airport Terminal)

In [36]:
ct_merged.loc[ct_merged['Cluster Labels'] == 1, ct_merged.columns[[1] + list(range(5, ct_merged.shape[1]))]]  

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
24,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
25,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
26,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
27,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
28,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
29,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport


#### Cluster 3 (Park)

In [37]:
ct_merged.loc[ct_merged['Cluster Labels'] == 2, ct_merged.columns[[1] + list(range(5, ct_merged.shape[1]))]]  

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Downtown Toronto,2,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner


#### Cluster 4 (Cafe / Food Places)

In [38]:
ct_merged.loc[ct_merged['Cluster Labels'] == 3, ct_merged.columns[[1] + list(range(5, ct_merged.shape[1]))]]  

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Lingerie Store,Pizza Place,Bar
3,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Lingerie Store,Pizza Place,Bar
8,Downtown Toronto,3,Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Bakery,Burger Joint,Gym,Hotel,Sushi Restaurant
9,Downtown Toronto,3,Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Bakery,Burger Joint,Gym,Hotel,Sushi Restaurant
10,Downtown Toronto,3,Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Bakery,Burger Joint,Gym,Hotel,Sushi Restaurant
18,Downtown Toronto,3,Café,Bar,Japanese Restaurant,Bookstore,Restaurant,Theater,Bakery,Coffee Shop,Beer Bar,Beer Store
19,Downtown Toronto,3,Café,Bar,Japanese Restaurant,Bookstore,Restaurant,Theater,Bakery,Coffee Shop,Beer Bar,Beer Store
20,Downtown Toronto,3,Café,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Farmers Market,Dumpling Restaurant
21,Downtown Toronto,3,Café,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Farmers Market,Dumpling Restaurant
22,Downtown Toronto,3,Café,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Farmers Market,Dumpling Restaurant


#### Cluster 5 (Coffee Shop)

In [39]:
ct_merged.loc[ct_merged['Cluster Labels'] == 4, ct_merged.columns[[1] + list(range(5, ct_merged.shape[1]))]]  

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Restaurant
1,Downtown Toronto,4,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Restaurant
4,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Breakfast Spot,Hotel,Bakery,Italian Restaurant,Gastropub,Park,Diner
5,Downtown Toronto,4,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Seafood Restaurant,Farmers Market,Steakhouse,Restaurant,Pub,Bakery
6,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Bar,Burger Joint,Bubble Tea Shop,Café,Sandwich Place,Chinese Restaurant,Sushi Restaurant,Middle Eastern Restaurant
11,Downtown Toronto,4,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Bakery,Fried Chicken Joint,Scenic Lookout,Pizza Place,Brewery
12,Downtown Toronto,4,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Bakery,Fried Chicken Joint,Scenic Lookout,Pizza Place,Brewery
13,Downtown Toronto,4,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Bakery,Fried Chicken Joint,Scenic Lookout,Pizza Place,Brewery
14,Downtown Toronto,4,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Deli / Bodega,Seafood Restaurant,Italian Restaurant,Steakhouse
15,Downtown Toronto,4,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Deli / Bodega,Seafood Restaurant,Italian Restaurant,Steakhouse
